In [30]:
import pandas as pd
from pathlib import Path

In [22]:
lbw = pd.read_csv('data/Natality_by_year_2007-2021.txt', sep="\t")
lbw.columns = [col.lower().replace(' ', '_') for col in lbw.columns]

In [29]:
lbw.drop(columns='notes').dropna().sort_values(by='year', ascending=False)

,state,state_code,county,county_code,infant_birth_weight_12,infant_birth_weight_12_code,year,year_code,births,%_of_total_births,average_birth_weight,average_lmp_gestational_age,average_oe_gestational_age
21213,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2000 - 2499 grams,5.0,2021.0,2021.0,230.0,0.01%,2303.27,36.19,36.30
11144,Missouri,29.0,"St. Charles County, MO",29183.0,2000 - 2499 grams,5.0,2021.0,2021.0,94.0,0.00%,2324.00,36.36,35.80
11120,Missouri,29.0,"St. Charles County, MO",29183.0,1000 - 1499 grams,3.0,2021.0,2021.0,20.0,0.00%,1277.05,30.30,30.05
1955,California,6.0,"San Diego County, CA",6073.0,1000 - 1499 grams,3.0,2021.0,2021.0,143.0,0.01%,1277.10,30.43,30.19
11108,Missouri,29.0,"St. Charles County, MO",29183.0,500 - 999 grams,2.0,2021.0,2021.0,10.0,0.00%,753.10,30.00,25.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7788,Iowa,19.0,"Unidentified Counties, IA",19999.0,500 - 999 grams,2.0,2007.0,2007.0,48.0,0.00%,785.85,26.13,25.72
15709,Ohio,39.0,"Warren County, OH",39165.0,1500 - 1999 grams,4.0,2007.0,2007.0,19.0,0.00%,1774.05,33.26,32.89
19108,Texas,48.0,"Travis County, TX",48453.0,500 - 999 grams,2.0,2007.0,2007.0,71.0,0.00%,759.96,25.44,25.31
15704,Ohio,39.0,"Warren County, OH",39165.0,1000 - 1499 grams,3.0,2007.0,2007.0,15.0,0.00%,1188.47,31.20,29.80


In [48]:
from pathlib import Path
data_path = Path('data/')
aqi_files = [file for file in data_path.glob('*aqi*') if file.is_file()]
aqi_files.sort()
aqi = pd.read_csv(aqi_files[0])
for file in aqi_files[1:]:
    aqi = pd.concat([aqi, pd.read_csv(file)])
aqi.columns = [col.lower().replace(' ', '_') for col in aqi.columns]
aqi.to_csv('data/aqi_by_year_2006-2021.csv', index=False)

In [49]:
aqi.sort_values(by='days_with_aqi').sample(50)

,state,county,year,days_with_aqi,good_days,moderate_days,unhealthy_for_sensitive_groups_days,unhealthy_days,very_unhealthy_days,hazardous_days,max_aqi,90th_percentile_aqi,median_aqi,days_co,days_no2,days_ozone,days_pm2.5,days_pm10
798,Pennsylvania,Northampton,2011,365,165,189,9,2,0,0,152,84,53,0,9,89,264,3
395,Maryland,Harford,2019,359,261,89,9,0,0,0,133,71,44,0,0,231,128,0
81,California,Napa,2010,365,293,70,1,1,0,0,159,60,39,0,3,184,178,0
853,South Carolina,Greenville,2006,365,118,240,5,2,0,0,159,82,59,0,16,0,348,1
279,Indiana,Henry,2016,119,106,13,0,0,0,0,65,51,28,0,0,0,119,0
775,Pennsylvania,Luzerne,2012,366,329,35,2,0,0,0,129,51,33,0,0,348,0,18
262,Illinois,McHenry,2009,365,242,123,0,0,0,0,100,67,43,0,0,174,191,0
98,California,Ventura,2021,365,208,147,10,0,0,0,122,78,48,0,0,278,75,12
402,Maine,Knox,2007,179,151,22,4,2,0,0,177,71,34,0,0,179,0,0
300,Iowa,Pottawattamie,2021,124,100,23,1,0,0,0,117,57,33,0,0,0,120,4
